In [ ]:
import pandas as pd
import numpy as np
import os

## Results from the paper

In [ ]:
GRU = {
    'mort_hosp':{'auc': 0.8504, 'auprc': 0.5215, 'F1': 0.4229},
    'mort_icu': {'auc': 0.8632, 'auprc': 0.4651, 'F1': 0.3630},
    'los_3':    {'auc': 0.6740, 'auprc': 0.6017, 'F1': 0.5336},
    'los_7':    {'auc': 0.7054, 'auprc': 0.1625, 'F1': 0.0233},
}

# Averaged Multimodal

word2vec_avg = {
    'mort_hosp':{'auc': 0.8642, 'auprc': 0.5422, 'F1': 0.4542},
    'mort_icu': {'auc': 0.8717, 'auprc': 0.4847, 'F1': 0.4230},
    'los_3':    {'auc': 0.6863, 'auprc': 0.6181, 'F1': 0.5419},
    'los_7':    {'auc': 0.7159, 'auprc': 0.1791, 'F1': 0.0135},
}

fastText_avg = {
    'mort_hosp':{'auc': 0.8609, 'auprc': 0.5447, 'F1': 0.4550},
    'mort_icu': {'auc': 0.8714, 'auprc': 0.4836, 'F1': 0.4291},
    'los_3':    {'auc': 0.6855, 'auprc': 0.6159, 'F1': 0.5446},
    'los_7':    {'auc': 0.7131, 'auprc': 0.1757, 'F1': 0.0102},
}

concat_avg = {
    'mort_hosp':{'auc': 0.8598, 'auprc': 0.5419, 'F1': 0.4566},
    'mort_icu': {'auc': 0.8690, 'auprc': 0.4828, 'F1': 0.4076},
    'los_3':    {'auc': 0.6861, 'auprc': 0.6169, 'F1': 0.5470},
    'los_7':    {'auc': 0.7159, 'auprc': 0.1767, 'F1': 0.0137},
}


# Proposed Model

word2vec_proposed = {
    'mort_hosp':{'auc': 0.8755, 'auprc': 0.5587, 'F1': 0.4723},
    'mort_icu': {'auc': 0.8835, 'auprc': 0.4923, 'F1': 0.4302},
    'los_3':    {'auc': 0.6954, 'auprc': 0.6268, 'F1': 0.5504},
    'los_7':    {'auc': 0.7255, 'auprc': 0.1878, 'F1': 0.0158},
}

fastText_proposed = {
    'mort_hosp':{'auc': 0.8715, 'auprc': 0.5568, 'F1': 0.4687},
    'mort_icu': {'auc': 0.8785, 'auprc': 0.4878, 'F1': 0.4309},
    'los_3':    {'auc': 0.6961, 'auprc': 0.6255, 'F1': 0.5587},
    'los_7':    {'auc': 0.7181, 'auprc': 0.1801, 'F1': 0.0108},
}

concat_proposed = {
    'mort_hosp':{'auc': 0.8698, 'auprc': 0.5535, 'F1': 0.4638},
    'mort_icu': {'auc': 0.8766, 'auprc': 0.4874, 'F1': 0.4224},
    'los_3':    {'auc': 0.6993, 'auprc': 0.6277, 'F1': 0.5582},
    'los_7':    {'auc': 0.7192, 'auprc': 0.1825, 'F1': 0.0138},
}


Place them into pandas df to be used later

In [ ]:
# paper_results_07
paper_results_07 = pd.DataFrame.from_dict(GRU, orient='index')
paper_results_07 = paper_results_07.reset_index()
paper_results_07.rename(columns={'index':'task'}, inplace=True)
paper_results_07['layer'] = 'GRU'

# paper_results_08
word2vec_avg_pd = pd.DataFrame.from_dict(word2vec_avg, orient='index')
word2vec_avg_pd = word2vec_avg_pd.reset_index()
word2vec_avg_pd.rename(columns={'index':'task'}, inplace=True)
word2vec_avg_pd['embedding'] = 'word2vec'

fastText_avg_pd = pd.DataFrame.from_dict(fastText_avg, orient='index')
fastText_avg_pd = fastText_avg_pd.reset_index()
fastText_avg_pd.rename(columns={'index':'task'}, inplace=True)
fastText_avg_pd['embedding'] = 'fasttext'

concat_avg = pd.DataFrame.from_dict(word2vec_avg, orient='index')
concat_avg = concat_avg.reset_index()
concat_avg.rename(columns={'index':'task'}, inplace=True)
concat_avg['embedding'] = 'concat'

paper_results_08 = pd.concat([word2vec_avg_pd, fastText_avg_pd, concat_avg])

# paper_results_09
word2vec_proposed_pd = pd.DataFrame.from_dict(word2vec_proposed, orient='index')
word2vec_proposed_pd = word2vec_proposed_pd.reset_index()
word2vec_proposed_pd.rename(columns={'index':'task'}, inplace=True)
word2vec_proposed_pd['embedding'] = 'word2vec'

fastText_proposed_pd = pd.DataFrame.from_dict(fastText_proposed, orient='index')
fastText_proposed_pd = fastText_proposed_pd.reset_index()
fastText_proposed_pd.rename(columns={'index':'task'}, inplace=True)
fastText_proposed_pd['embedding'] = 'fasttext'

concat_proposed_pd = pd.DataFrame.from_dict(concat_proposed, orient='index')
concat_proposed_pd = concat_proposed_pd.reset_index()
concat_proposed_pd.rename(columns={'index':'task'}, inplace=True)
concat_proposed_pd['embedding'] = 'concat'

paper_results_09 = pd.concat([word2vec_avg_pd, fastText_avg_pd, concat_avg])


## Functions to produce tabular outputs

In [ ]:
# Add n blank rows
def add_blank_rows_df(df, after_each_n):
    new_index = pd.RangeIndex(len(df) + np.ceil(len(df)/after_each_n))
    new_df = pd.DataFrame('', index=new_index, columns=df.columns)
    
    ids = []
    jump = 0
    for i in range(0, len(df)):
        ids.append(i + np.floor(i/after_each_n))
        
    new_df.loc[ids] = df.values
    return new_df

def get_results(result_path: str,
                paper_results,
                model_category: str,
                sort_by:str,
                group_by: list,
                exp_naming: list,
                var_type: str):
    
    result_dists = []
    exps = os.listdir(result_path)
    
    # Pulls the results from the `.p` files 
    for exp in exps:
        if '.p' not in exp:
            continue
        result_dict = pd.read_pickle(os.path.join(result_path, exp))
        result_dict[exp_naming[0]], result_dict[exp_naming[1]], result_dict[exp_naming[2]], result_dict[exp_naming[3]] = exp.split("-")[:4]
        result_dict["model_category"] = model_category
        result_dists.append(result_dict)
        
    result = pd.DataFrame(result_dists)
    result.sort_values(by=['task', sort_by], inplace=True, ascending=False)

    # Generate the summary results table
    summary_results = get_summary_results(result, 
                                          model_category,
                                          sort_by,
                                          group_by,
                                          exp_naming,
                                          var_type)
    
    
    results_difference = get_results_difference(result,
                                                paper_results,
                                                model_category,
                                                sort_by,
                                                group_by,
                                                exp_naming,
                                                var_type)
    
    return result, summary_results, results_difference

def get_summary_results(result, 
                        model_category: str,
                        sort_by:str,
                        group_by: list,
                        exp_naming: list,
                        var_type: str):
    
    summary_results = result.groupby(by=group_by).agg({"auc":['mean','std'], "auprc":['mean','std'], "F1":['mean','std'], 'acc':['mean','std']})
    summary_results.columns = summary_results.columns.map('_'.join).str.strip('_')
    summary_results.reset_index(inplace=True)
    summary_results["model_category"] = model_category
    
    # Get values in terms of %
    for col in ['auc_mean', 'auc_std', 'auprc_mean', 'auprc_std',
       'F1_mean', 'F1_std', 'acc_mean', 'acc_std']:
       summary_results[col] = summary_results[col] * 100
    
    # Rename the tasks
    task_mapping = {
        "los_3": "LOS > 3 Days",
        "los_7": "LOS > 7 Days",
        "mort_hosp": "In-Hospital Mortality",
        "mort_icu": "In-ICU Mortality"
    }
    for k, v in task_mapping.items():
        summary_results.loc[summary_results["task"] == k, "task"] = v
    
    # Format the numeric results
    for matric in ["auc", "auprc", "F1", "acc"]:
        summary_results[matric] = summary_results[[f"{matric}_mean", f"{matric}_std"]].apply(lambda x: "{:.2f} +/- {:.3f}".format(x[0], x[1]), axis=1)
    
    # Order the results in the manner desired
    summary_results = summary_results[["task", "model_category",var_type, "auc", "auprc", "F1"]]

    summary_results.sort_values(by=[sort_by], inplace=True, ascending=False)
    summary_results.sort_values(by=['task'], inplace=True, ascending=True)

    return summary_results


def get_results_difference(result, 
                           paper_results,
                           model_category: str,
                           sort_by:str,
                           group_by: list,
                           exp_naming: list,
                           var_type: str):
    
    results_difference = result.groupby(by=group_by).agg({"auc":['mean'], "auprc":['mean'], "F1":['mean']})
    results_difference = results_difference.rename(columns={'mean':''})
    results_difference.columns = results_difference.columns.map('_'.join).str.strip('_')
    results_difference.reset_index(inplace=True)
    results_difference["model_category"] = model_category
    
    # Take the difference between the reported results in the paper and what we have recorded [auc, auprc, F1]
    results_difference = results_difference.assign(auc = results_difference['auc'] - results_difference.merge(paper_results, 
                                                                                         on=['task',var_type], 
                                                                                         suffixes=('','_y'), 
                                                                                         how='left')['auc_y'])  
    results_difference = results_difference.assign(auprc = results_difference['auprc'] - results_difference.merge(paper_results, 
                                                                                         on=['task',var_type], 
                                                                                         suffixes=('','_y'), 
                                                                                         how='left')['auprc_y']) 
    results_difference = results_difference.assign(F1 = results_difference['F1'] - results_difference.merge(paper_results, 
                                                                                         on=['task',var_type], 
                                                                                         suffixes=('','_y'), 
                                                                                         how='left')['F1_y']) 
    # If no data from paper, then set to '-'
    results_difference.loc[results_difference.merge(paper_results, 
                                                     on=['task',var_type], 
                                                     suffixes=('','_y'), 
                                                     how='left')['auc_y'].isnull(), 'auc'] = '-'
    results_difference.loc[results_difference.merge(paper_results, 
                                                     on=['task',var_type], 
                                                     suffixes=('','_y'), 
                                                     how='left')['auprc_y'].isnull(), 'auprc'] = '-'
    results_difference.loc[results_difference.merge(paper_results, 
                                                     on=['task',var_type], 
                                                     suffixes=('','_y'), 
                                                     how='left')['F1_y'].isnull(), 'F1'] = '-'


    # Get values in terms of %
    for col in ['auc', 'auprc', 'F1']:
       results_difference[col] = results_difference[col] * 100
    
    # Rename the tasks
    task_mapping = {
        "los_3": "LOS > 3 Days",
        "los_7": "LOS > 7 Days",
        "mort_hosp": "In-Hospital Mortality",
        "mort_icu": "In-ICU Mortality"
    }
    for k, v in task_mapping.items():
        results_difference.loc[results_difference["task"] == k, "task"] = v
    
    # Format the numeric results
    for metric in ["auc", "auprc", "F1"]:
        results_difference[metric] = results_difference[[metric]].apply(lambda x: "{:.2f}".format(x[0]) if not isinstance(x[0], str) else '-', axis=1)
    
    # Order the results in the manner desired
    results_difference = results_difference[["task", "model_category",var_type, "auc", "auprc", "F1"]]
    
    results_difference.sort_values(by=[sort_by], inplace=True, ascending=False)
    results_difference.sort_values(by=['task'], inplace=True, ascending=True)
    
    return results_difference


# Only time-series data results

In [ ]:
results_07, summary_results_07, results_difference_07 = get_results(
    result_path="results/07-baseline/GRU/hidden_units_256",
    paper_results=paper_results_07,
    model_category="GRU",
    sort_by="auc",
    group_by=["task", "layer"],
    exp_naming=["hidden_units","layer","task","iteration"],
    var_type="layer")

summary_results_07

In [ ]:
results_difference_07

# Baseline time-series and NLP data results

In [ ]:
results_08, summary_results_08, results_difference_08 = get_results(
    result_path="results/08-multimodal",
    paper_results=paper_results_08,
    model_category="Average Multimodal",
    sort_by="auc",
    group_by=["task", "embedding"],
    exp_naming=["layer","hidden_units","embedding","task"],
    var_type="embedding")

summary_results_08_print = add_blank_rows_df(summary_results_08, 5)
summary_results_08_print

In [ ]:
results_difference_08_print = add_blank_rows_df(results_difference_08, 5)
results_difference_08_print

# Proposed models

In [ ]:
results_09, summary_results_09, results_difference_09 = get_results(
    result_path="results/09-cnn",
    paper_results=paper_results_09,
    model_category="Proposed Model",
    sort_by="embedding",
    group_by=["task", "embedding"],
    exp_naming=["sequence_name","hidden_unit_size","embedding","task"],
    var_type="embedding")

summary_results_09_print = add_blank_rows_df(summary_results_09, 5)
summary_results_09_print

In [ ]:
results_difference_09_print = add_blank_rows_df(results_difference_09, 5)
results_difference_09_print

# Make same as paper (Table 3)

In [ ]:
def table_3(df1, df2):

    table_3_df = pd.concat([df1, df2])
    table_3_df = table_3_df.drop(columns=['layer']).fillna('')
    table_3_df = table_3_df[["task", "model_category",'embedding', "auc", "auprc", "F1"]]
    table_3_df.sort_values(by=['task', 'embedding'], inplace=True, ascending=True)
    table_3_df = add_blank_rows_df(table_3_df, 6)
    return table_3_df

summary_results_paper_1 = table_3(summary_results_07, summary_results_08)
summary_results_paper_1

In [ ]:
results_difference_paper_1 = table_3(results_difference_07, results_difference_08)
results_difference_paper_1

## Table 4

### Determine best model from table 3

In [ ]:
tasks = ["los_3","los_7","mort_hosp", "mort_icu" ]

best_baseline = pd.concat([results_07, results_08]).fillna('-')

best_baseline = best_baseline.groupby(by=['task', 'embedding']).agg({"auc":['mean', 'std'], 
                                                                     "auprc":['mean', 'std'],
                                                                     "F1":['mean', 'std']})
best_baseline.columns = best_baseline.columns.map('_'.join).str.strip('_')
best_baseline.reset_index(inplace=True)

best_baseline_auc = best_baseline.loc[best_baseline.groupby("task")["auc_mean"].idxmax()]
best_baseline_auc = best_baseline_auc[["task", "auc_mean", 'auc_std']]

best_baseline_auprc = best_baseline.loc[best_baseline.groupby("task")["auprc_mean"].idxmax()]
best_baseline_auprc = best_baseline_auprc[["task", "auprc_mean", 'auprc_std']]

best_baseline_F1 = best_baseline.loc[best_baseline.groupby("task")["F1_mean"].idxmax()]
best_baseline_F1 = best_baseline_F1[["task", "F1_mean", 'F1_std']]

best_baseline = best_baseline_auc.merge(best_baseline_auprc, on='task').merge(best_baseline_F1, on='task')


# Get values in terms of %
for col in ['auc_mean', 'auc_std', 'auprc_mean', 'auprc_std', 'F1_mean', 'F1_std']:
   best_baseline[col] = best_baseline[col] * 100

# Rename the tasks
task_mapping = {
    "los_3": "LOS > 3 Days",
    "los_7": "LOS > 7 Days",
    "mort_hosp": "In-Hospital Mortality",
    "mort_icu": "In-ICU Mortality"
}
for k, v in task_mapping.items():
    best_baseline.loc[best_baseline["task"] == k, "task"] = v

# Format the numeric results
for metric in ["auc", "auprc", "F1"]:
    best_baseline[metric] = best_baseline[[f"{metric}_mean", f"{metric}_std"]].apply(lambda x: "{:.2f} +/- {:.3f}".format(x[0], x[1]), axis=1)

best_baseline["model_category"] = 'Best Baseline'
best_baseline["embedding"] = '-'

# Order the results in the manner desired
best_baseline = best_baseline[["task", "model_category", 'embedding', "auc", "auprc", "F1"]]

best_baseline

### Add best baseline to CNN results

In [ ]:
def table_4(df1, df2):

    table_3_df = pd.concat([df1, df2])
#     table_3_df = table_3_df.drop(columns=['layer']).fillna('')
    table_3_df = table_3_df[["task", "model_category",'embedding', "auc", "auprc", "F1"]]
    table_3_df.sort_values(by=['task', 'model_category', 'embedding'], inplace=True, ascending=True)
    table_3_df = add_blank_rows_df(table_3_df, 6)
    return table_3_df

summary_results_paper_2 = table_4(best_baseline, summary_results_09)
summary_results_paper_2

In [ ]:
# results_difference_paper_2 = table_4(results_difference_07, results_difference_08)
# results_difference_paper_2

In [ ]:
# print(summary_results_paper_1.style.to_latex())

In [ ]:
# print(summary_results_paper_2.style.to_latex())